In [1]:
import torch
import torch.nn as nn


In [22]:
import torch
import torch.nn as nn

class Model(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_features, 3),
        nn.ReLU(),
        nn.Linear(3, 1),
        nn.Sigmoid()
    )

  def forward(self, features):
    out = self.network(features)  # z = w * x + b

    return out

In [23]:
# create dataset
features = torch.rand(10, 5) # 5 columns => 5 features

# create model
model = Model(features.shape[1])

# forward pass
model(features) # good practice for forward pass instead of using model.forward(). the forward() method gets triggered automatically



tensor([[0.3735],
        [0.3817],
        [0.3777],
        [0.3631],
        [0.3346],
        [0.3664],
        [0.3313],
        [0.3696],
        [0.3610],
        [0.3356]], grad_fn=<SigmoidBackward0>)

In [18]:
# show model weights
model.linear1.weight

Parameter containing:
tensor([[-0.1573,  0.2806,  0.1796,  0.1689, -0.1970],
        [ 0.0197,  0.2739,  0.2753,  0.0968,  0.0570],
        [ 0.0163, -0.3216, -0.3464,  0.3907,  0.1363]], requires_grad=True)

In [20]:
model.linear1.bias

Parameter containing:
tensor([-0.2318, -0.3745,  0.2310], requires_grad=True)

In [14]:
!pip install torchinfo

In [21]:
from torchinfo import summary

summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

Improving the Breast Cancer Detection Training Pipeline

In [24]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [25]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [26]:
df.shape

(569, 33)

In [27]:
df.drop(['id','Unnamed: 32'], axis=1, inplace=True)

In [28]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [29]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)


In [30]:
# scaling: adjusts values to similar range

scaler = StandardScaler()

# computes the mean and std form training data
# Then transforms training data
x_train = scaler.fit_transform(X_train)
# never use .fit() on test data — it would "leak" information from test set into training.
x_test = scaler.transform(X_test)

In [31]:
x_train

array([[-0.38297728,  0.74496914, -0.41879044, ..., -0.31979128,
        -0.09396474, -0.15595142],
       [ 0.68124376,  2.0763311 ,  0.65189878, ...,  0.42070501,
        -1.0854502 , -0.293711  ],
       [-0.32462574,  0.00453254, -0.30600151, ..., -0.23931251,
         1.15066594,  0.45699506],
       ...,
       [-0.39687051, -1.31733664, -0.41637353, ..., -0.77609215,
        -0.8404186 , -0.79455353],
       [ 0.58399118,  0.04250365,  0.71232142, ...,  0.96588377,
         0.69954817,  0.44026312],
       [ 1.75657938,  0.31779418,  1.71533728, ...,  0.71849173,
         0.54052104, -1.0934974 ]])

In [32]:
y_train
# Theh y_train contains alphabetical chars so our neural network won't
# be able to recognize it. So we've to encode it using LabelEncoder()

,diagnosis
135,M
566,M
331,B
372,M
164,M
...,...
90,B
531,B
294,B
479,M


In [51]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [52]:
y_train

array([1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,

In [92]:
# convert numpy arrays to tensors
X_train_tensor = torch.from_numpy(x_train).float()
X_test_tensor = torch.from_numpy(x_test).float()
y_train_tensor = torch.from_numpy(y_train).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [93]:
X_train_tensor

tensor([[-0.3830,  0.7450, -0.4188,  ..., -0.3198, -0.0940, -0.1560],
        [ 0.6812,  2.0763,  0.6519,  ...,  0.4207, -1.0855, -0.2937],
        [-0.3246,  0.0045, -0.3060,  ..., -0.2393,  1.1507,  0.4570],
        ...,
        [-0.3969, -1.3173, -0.4164,  ..., -0.7761, -0.8404, -0.7946],
        [ 0.5840,  0.0425,  0.7123,  ...,  0.9659,  0.6995,  0.4403],
        [ 1.7566,  0.3178,  1.7153,  ...,  0.7185,  0.5405, -1.0935]])

Simple NN

In [94]:
class MySimpleNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [95]:
learning_rate = 0.1
epochs = 25

In [96]:
loss_function = nn.BCELoss()

Create Model

In [98]:
model = MySimpleNN(X_train_tensor.shape[1])

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
  # forward pass
  y_pred = model(X_train_tensor)

  # loss function
  loss = loss_function(y_pred, y_train_tensor.view(-1, 1))

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f"Epoch: {epoch + 1}, Loss: {loss}")

Epoch: 1, Loss: 0.7648435235023499
Epoch: 2, Loss: 0.5707852244377136
Epoch: 3, Loss: 0.47181573510169983
Epoch: 4, Loss: 0.4126967191696167
Epoch: 5, Loss: 0.37264299392700195
Epoch: 6, Loss: 0.3432493805885315
Epoch: 7, Loss: 0.32048746943473816
Epoch: 8, Loss: 0.30216944217681885
Epoch: 9, Loss: 0.28699633479118347
Epoch: 10, Loss: 0.27414411306381226
Epoch: 11, Loss: 0.2630622684955597
Epoch: 12, Loss: 0.25336816906929016
Epoch: 13, Loss: 0.24478647112846375
Epoch: 14, Loss: 0.23711363971233368
Epoch: 15, Loss: 0.23019558191299438
Epoch: 16, Loss: 0.22391334176063538
Epoch: 17, Loss: 0.21817319095134735
Epoch: 18, Loss: 0.21290038526058197
Epoch: 19, Loss: 0.20803436636924744
Epoch: 20, Loss: 0.20352545380592346
Epoch: 21, Loss: 0.19933244585990906
Epoch: 22, Loss: 0.19542080163955688
Epoch: 23, Loss: 0.19176124036312103
Epoch: 24, Loss: 0.18832874298095703
Epoch: 25, Loss: 0.18510191142559052


In [99]:
model.linear.weight

Parameter containing:
tensor([[ 0.3234,  0.3019,  0.3243,  0.2049,  0.1139,  0.1029,  0.2279,  0.3178,
          0.1322, -0.1289,  0.0617, -0.1109,  0.0236,  0.3321, -0.0540, -0.1601,
          0.0942,  0.2039,  0.0007, -0.0781,  0.3940,  0.2149,  0.1399,  0.0969,
          0.1272,  0.1797,  0.2615,  0.1528,  0.0465,  0.0862]],
       requires_grad=True)

Model Evaluation

In [100]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  # print(y_pred)
  # print(y_test_tensor)
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.5507848858833313
